<a href="https://colab.research.google.com/github/Apache163/Cars/blob/main/Task12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Math
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Формула для расчета изменения температуры (изображение)
display(Math(r'\Delta T = \frac{V_0 \cdot \Delta P + P_0 \cdot \Delta V}{nR}'))

# Константы
V0 = 15.0  # дм3
P0 = 100.0  # Па
T0 = 280.0  # K
nR = 8.314  # Универсальная газовая постоянная (упрощенно)

# Генерация синтетических данных для обучения
def generate_data(num_samples=1000):
    np.random.seed(42)
    dP = np.random.uniform(-10, 10, num_samples)  # Изменение давления
    dV = np.random.uniform(-1, 1, num_samples)    # Изменение объема

    # Расчет истинного изменения температуры по формуле
    dT = (V0 * dP + P0 * dV) / nR

    return np.column_stack((dP, dV)), dT

# Создаем и обучаем модель
X, y = generate_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

# Элементы интерфейса
dP_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='ΔP (Па):')
dV_slider = widgets.FloatSlider(value=0, min=-1, max=1, step=0.01, description='ΔV (дм3):')
predict_button = widgets.Button(description="Рассчитать ΔT")
train_button = widgets.Button(description="Обучить модель")
output = widgets.Output()
plot_output = widgets.Output()

# График для визуализации
fig, ax = plt.subplots(figsize=(10, 5))
plt.close(fig)

def calculate_true_dT(dP, dV):
    """Расчет по физической формуле"""
    return (V0 * dP + P0 * dV) / nR

def on_predict_button_clicked(b):
    with output:
        output.clear_output()
        dP = dP_slider.value
        dV = dV_slider.value

        # Масштабирование входных данных
        input_scaled = scaler.transform([[dP, dV]])

        # Предсказание модели
        nn_dT = model.predict(input_scaled)[0]

        # Расчет по формуле
        true_dT = calculate_true_dT(dP, dV)

        print(f"Изменение давления (ΔP): {dP:.2f} Па")
        print(f"Изменение объема (ΔV): {dV:.2f} дм3")
        print(f"Предсказанное ΔT (нейросеть): {nn_dT:.4f} K")
        print(f"Расчетное ΔT (формула): {true_dT:.4f} K")
        print(f"Разница: {abs(nn_dT - true_dT):.4f} K")

    with plot_output:
        plot_output.clear_output()

        # Визуализация
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        # График зависимости ΔT от ΔP при фиксированном ΔV
        dP_range = np.linspace(-10, 10, 100)
        current_dV = dV_slider.value
        input_scaled = scaler.transform(np.column_stack((dP_range, np.full_like(dP_range, current_dV))))
        pred_dT = model.predict(input_scaled)
        true_dT = calculate_true_dT(dP_range, current_dV)

        ax1.plot(dP_range, pred_dT, label='Нейросеть')
        ax1.plot(dP_range, true_dT, '--', label='Формула')
        ax1.set_xlabel('ΔP (Па)')
        ax1.set_ylabel('ΔT (K)')
        ax1.set_title(f'Зависимость ΔT от ΔP при ΔV={current_dV:.2f}')
        ax1.legend()
        ax1.grid()

        # График зависимости ΔT от ΔV при фиксированном ΔP
        dV_range = np.linspace(-1, 1, 100)
        current_dP = dP_slider.value
        input_scaled = scaler.transform(np.column_stack((np.full_like(dV_range, current_dP), dV_range)))
        pred_dT = model.predict(input_scaled)
        true_dT = calculate_true_dT(current_dP, dV_range)

        ax2.plot(dV_range, pred_dT, label='Нейросеть')
        ax2.plot(dV_range, true_dT, '--', label='Формула')
        ax2.set_xlabel('ΔV (дм3)')
        ax2.set_ylabel('ΔT (K)')
        ax2.set_title(f'Зависимость ΔT от ΔV при ΔP={current_dP:.2f}')
        ax2.legend()
        ax2.grid()

        plt.tight_layout()
        plt.show()

def on_train_button_clicked(b):
    with output:
        output.clear_output()
        print("Переобучение модели...")
        global model
        model = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)
        model.fit(X_train_scaled, y_train)

        # Оценка качества
        train_score = model.score(X_train_scaled, y_train)
        test_score = model.score(X_test_scaled, y_test)

        print(f"R² на обучающей выборке: {train_score:.4f}")
        print(f"R² на тестовой выборке: {test_score:.4f}")
        print("Модель переобучена!")

predict_button.on_click(on_predict_button_clicked)
train_button.on_click(on_train_button_clicked)

# Отображаем интерфейс
display(widgets.VBox([
    widgets.HTML("<h3>Нейросетевой регулятор температуры</h3>"),
    widgets.HTML(f"Начальные условия: V₀={V0} дм³, P₀={P0} Па, T₀={T0} K"),
    dP_slider,
    dV_slider,
    widgets.HBox([predict_button, train_button]),
    output,
    plot_output
]))